In [1]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
project_dir = "/data/ephemeral/home/nlp-5/song/"
import os
import sys
sys.path.append(
    project_dir
)
from src.utils.utils import *

import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

train_df = pd.read_csv(os.path.join(project_dir, 'data','train.csv'))
val_df = pd.read_csv(os.path.join(project_dir, 'data','dev.csv'))

test_df = pd.read_csv(os.path.join(project_dir, 'data','test.csv'))
sub_df = pd.read_csv(os.path.join(project_dir, 'outputs', 'exp_aug_0730191312', 'submission_0730191312-0.csv'))

In [3]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords') # nltk 내장 stopword 다운로드

[nltk_data] Downloading package punkt to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### 불용어 분석 
- 토큰의 빈도를 분석하여 불용어 판단

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization")
special_tokens = [
    '#Person1#','#Person2#','#Person3#','#Person4#','#Person5#','#Person6#','#Person7#',
    '#PhoneNumber#','#Address#','#DateOfBirth#','#PassportNumber#','#SSN#','#CardNumber#','#CarNumber#','#Email#'
]
special_tokens_dict={'additional_special_tokens':special_tokens}
tokenizer.add_special_tokens(special_tokens_dict)

17

In [16]:
from collections import Counter
from tqdm import tqdm
all_dialogues = pd.concat([train_df['dialogue'], val_df['dialogue'], test_df['dialogue']])
all_tokens = []

for text in tqdm(all_dialogues):
    # 토크나이저를 사용하여 텍스트를 토큰화합니다.
    # .tokenize() 메서드는 문자열 토큰 리스트를 반환합니다.
    tokens = tokenizer.tokenize(text)
    all_tokens.extend(tokens)
len_all_tokens = len(all_tokens)
# 4. 토큰 빈도 계산
token_counts = Counter(all_tokens)

100%|██████████| 13455/13455 [00:02<00:00, 4759.09it/s]


In [18]:
# 5. 가장 빈도가 높은 50개 토큰 출력
print("가장 빈도가 높은 상위 50개 토큰:")
token_counts_df = []
for token, count in token_counts.items():
    token = tokenizer.convert_tokens_to_string([token])
    # print(f"토큰: '{token}', 빈도: {count}, 비율: {count/len_all_tokens*100:.3f}%")
    token_counts_df.append([token, count, count/len_all_tokens*100])
token_counts_df = pd.DataFrame(
    data=token_counts_df,
    columns=['token', 'count', 'ratio']
)
token_counts_df = token_counts_df.sort_values('count', ascending=False)
token_counts_df = token_counts_df.reset_index(drop=True)
token_counts_df.head(50)

가장 빈도가 높은 상위 50개 토큰:


,token,count,ratio
0,:,127675,6.239392
1,#Person1#,66318,3.240917
2,#Person2#,60997,2.980883
3,.,39118,1.911671
4,",",36281,1.773028
5,\n,25071,1.225203
6,.\n,19033,0.930130
7,요.,17515,0.855946
8,?\n,14325,0.700053
9,?,12970,0.633835


- special token 추가
    - `\n`

In [19]:
tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization")
special_tokens = [
    '#Person1#','#Person2#','#Person3#','#Person4#','#Person5#','#Person6#','#Person7#',
    '#PhoneNumber#','#Address#','#DateOfBirth#','#PassportNumber#','#SSN#','#CardNumber#','#CarNumber#','#Email#',
    '\n'
]
special_tokens_dict={'additional_special_tokens':special_tokens}
tokenizer.add_special_tokens(special_tokens_dict)

16

In [38]:
from collections import Counter
from tqdm import tqdm
all_dialogues = pd.concat([train_df['dialogue'], val_df['dialogue'], test_df['dialogue']])
all_tokens = []

for text in tqdm(all_dialogues):
    # 토크나이저를 사용하여 텍스트를 토큰화합니다.
    # .tokenize() 메서드는 문자열 토큰 리스트를 반환합니다.
    tokens = tokenizer.tokenize(text)
    all_tokens.extend(tokens)
# 4. 토큰 빈도 계산
token_counts = Counter(all_tokens)

100%|██████████| 13455/13455 [00:02<00:00, 4688.34it/s]


In [41]:
# 5. 가장 빈도가 높은 50개 토큰 출력
print("가장 빈도가 높은 상위 50개 토큰:")
token_counts_df = []
len_token_counts = sum(token_counts.values())
for token, count in token_counts.items():
    token = tokenizer.convert_tokens_to_string([token])
    # print(f"토큰: '{token}', 빈도: {count}, 비율: {count/len_all_tokens*100:.3f}%")
    token_counts_df.append([token, count, count/len_token_counts*100])
token_counts_df = pd.DataFrame(
    data=token_counts_df,
    columns=['token', 'count', 'ratio']
)
token_counts_df = token_counts_df.sort_values('count', ascending=False)
token_counts_df = token_counts_df.reset_index(drop=True)
token_counts_df.head(50)

가장 빈도가 높은 상위 50개 토큰:


,token,count,ratio
0,:,127675,5.874355
1,\n,114131,5.251193
2,#Person1#,66318,3.051306
3,#Person2#,60997,2.806486
4,.,55155,2.537694
5,",",36282,1.669343
6,?,30658,1.410581
7,,30176,1.388405
8,요.,27788,1.278532
9,요?,19908,0.915972


### 영단어 분석

- 2개의 연속된 영단어 검출
    - 영어 이름 검색 가능

In [54]:
from collections import Counter
from tqdm import tqdm
import re

all_dialogues = pd.concat([train_df['dialogue'], val_df['dialogue'], test_df['dialogue']])
all_english = []

# 반복문을 돌며 영어 단어들을 수집하여 all_english 에 추가
for text in tqdm(all_dialogues):
    two_word_phrases = re.findall(r'\b(?:[a-zA-Z]+\.?\s+[a-zA-Z]+)\b', text, re.IGNORECASE)
    all_english.extend(two_word_phrases)
eng_counts = Counter(all_english)

len_eng_counts = sum(eng_counts.values())
word_counts_df = []
for eng_word, count in eng_counts.items():
    word_counts_df.append([eng_word, count, count/len_eng_counts*100])
word_counts_df = pd.DataFrame(
    data=word_counts_df,
    columns=['word', 'count', 'ratio']
)
word_counts_df = word_counts_df.sort_values('count', ascending=False)
word_counts_df = word_counts_df.reset_index(drop=True)
print(word_counts_df.shape[0])
word_counts_df.head(50)

100%|██████████| 13455/13455 [00:00<00:00, 77399.48it/s]

1030


,word,count,ratio
0,Mr. Smith,49,2.820956
1,Mr. Brown,43,2.475533
2,Mr. White,32,1.842257
3,Mr. Green,28,1.611975
4,Mr. Jones,23,1.324122
5,Mrs. Smith,22,1.266552
6,Mr. Black,18,1.036269
7,Mr. Taylor,12,0.690846
8,Mr. Li,10,0.575705
9,Mr. Johnson,10,0.575705


In [49]:
contain_df = train_df[train_df['dialogue'].str.contains('excuse me')]
print(len(contain_df))
print(contain_df['dialogue'].values[0])
print(contain_df['summary'].values[0])

1
#Person1#: 스미스 씨, 질문 하나 해도 될까요?
#Person2#: 네, 말씀하세요.
#Person1#: 꽤 오랫동안 'I'm sorry'랑 'excuse me'라는 두 표현의 차이가 헷갈렸어요. 중국어로 번역하면 같은 의미처럼 보이거든요. 어떻게 올바르게 사용하는지 설명해 주실 수 있을까요?
#Person2#: 좋아요, 사실 이 둘 사이에는 큰 차이가 있어요. 몇 가지 예를 들어 설명해 드릴게요. 누군가와 이야기 중에 대화를 끝내거나 자리를 비울 때 공손하게 말할 수 있는 표현이에요.
#Person1#: 그게 'I'm sorry'인가요?
#Person2#: 아니요, 'excuse me' 혹은 'excuse me, please'라고 말해요. 그리고 다른 사람의 말을 끊을 때도 'excuse me'를 사용해요...
#Person1#: 실례합니다, 스미스 씨.
#Person2#: 아주 잘하셨어요, 양 씨. 무슨 말씀하시려고 했나요?
#Person1#: 만약 우연히 누군가에게 부딪히거나 지나가려 할 때는 'sorry'나 'I'm so sorry'라고 해야 하나요?
#Person2#: 정말 빠르게 배우시네요.
양 씨는 'I'm sorry'와 'excuse me'라는 표현의 차이에 대해 혼란스러워 했습니다. 스미스 씨는 이 두 표현을 어떻게 올바르게 사용하는지 설명합니다.


In [50]:
contain_df = test_df[test_df['dialogue'].str.contains('excuse me')]
print(len(contain_df))
print(contain_df['dialogue'].values[0])

0


IndexError: index 0 is out of bounds for axis 0 with size 0

In [55]:
word_counts_df.to_csv(os.path.join(project_dir,'data','01_two_words_english_counts.csv'),index=False)

- 1개 영단어 검출

In [56]:
from collections import Counter
from tqdm import tqdm
import re

all_dialogues = pd.concat([train_df['dialogue'], val_df['dialogue'], test_df['dialogue']])
all_english = []

# 반복문을 돌며 영어 단어들을 수집하여 all_english 에 추가
for text in tqdm(all_dialogues):
    all_potential_words = re.findall(r'\b[a-zA-Z\']+\b', text, re.IGNORECASE)
    all_english.extend(all_potential_words)
eng_counts = Counter(all_english)

len_eng_counts = sum(eng_counts.values())
word_counts_df = []
for eng_word, count in eng_counts.items():
    word_counts_df.append([eng_word, count, count/len_eng_counts*100])
word_counts_df = pd.DataFrame(
    data=word_counts_df,
    columns=['word', 'count', 'ratio']
)
word_counts_df = word_counts_df.sort_values('count', ascending=False)
word_counts_df = word_counts_df.reset_index(drop=True)
print(word_counts_df.shape[0])
word_counts_df.head(50)

100%|██████████| 13455/13455 [00:00<00:00, 65675.65it/s]

2545


,word,count,ratio
0,Mr,797,6.439884
1,',484,3.910795
2,TV,249,2.011959
3,PhoneNumber,238,1.923077
4,Mary,202,1.632191
5,Smith,152,1.228184
6,Mrs,150,1.212023
7,Tom,142,1.147382
8,John,130,1.050420
9,Bob,104,0.840336


In [57]:
word_counts_df.to_csv(os.path.join(project_dir,'data','01_one_words_english_counts.csv'),index=False)

- 1개 이상 연속된 영단어 모두 검출

In [104]:
from collections import Counter
from tqdm import tqdm
import regex

all_dialogues = pd.concat([train_df['dialogue'], val_df['dialogue'], test_df['dialogue']])
all_english = []

# 반복문을 돌며 영어 단어들을 수집하여 all_english 에 추가
for text in tqdm(all_dialogues):
    # two_word_phrases = re.findall(r'\b[a-zA-Z\']+\b(?:[^\w\s]*\s*\b[a-zA-Z\']+\b){2,}', text, re.IGNORECASE)
    every_word_phrases = regex.findall(r"[\p{Latin}\.']+(?: [\p{Latin}\.']+)*", text, re.IGNORECASE)
    for eng_word in every_word_phrases:
        if eng_word.endswith('.'): # 영단어가 마침표로 끝나면 마침표만 제거
            eng_word = eng_word.replace('.','')
        
    all_english.extend(every_word_phrases)
eng_counts = Counter(all_english)

len_eng_counts = sum(eng_counts.values())
word_counts_df = []
for eng_word, count in eng_counts.items():
    word_counts_df.append([eng_word, count, count/len_eng_counts*100])
word_counts_df = pd.DataFrame(
    data=word_counts_df,
    columns=['word', 'count', 'ratio']
)
word_counts_df = word_counts_df.sort_values('count', ascending=False)
word_counts_df = word_counts_df.reset_index(drop=True)
print(word_counts_df.shape[0])
word_counts_df.tail(50)

100%|██████████| 13455/13455 [00:00<00:00, 44370.40it/s]

4734


,word,count,ratio
4684,... Vanilla Ice,1,0.000335
4685,'Ice Ice Baby',1,0.000335
4686,Mr. Liang,1,0.000335
4687,. DKNY,1,0.000335
4688,Grant,1,0.000335
4689,PHS,1,0.000335
4690,Mrs. Word,1,0.000335
4691,. Stanley,1,0.000335
4692,Sterling,1,0.000335
4693,Professor Wood,1,0.000335


In [107]:
contain_df = test_df[test_df['dialogue'].str.contains("FICA")]
print(len(contain_df))
print(contain_df['dialogue'].values[0])

1
#Person1#: 안녕, Susan. 잠깐 시간 있어? 내 급여에 대해 궁금한 게 좀 있어서. 
#Person2#: 물론이지, Emily. 앉아서 얘기해. 
#Person1#: 음, 여기 미국에서 받은 첫 급여인데 이해가 안 가는 부분이 몇 가지 있어. 우선, FICA랑 SUI 세금이 뭐고, 왜 Medicare랑 건강 보험 계획 둘 다에서 공제가 되는 거야? 
#Person2#: 좋아, 급여 명세서의 맨 위부터 시작하자. 이 숫자는 네 총 급여를 나타내. 그다음에는 일련의 공제가 있어. 먼저 연방 공제부터 볼게. FICA는 Federal Insurance Contribution Act의 약자야. 이게 네 연방 소득세야. 그리고 사회보장세와 Medicare가 있는데, 이 두 개는 네가 은퇴하거나 일을 못 하게 됐을 때를 지원하기 위한 연방 프로그램이야. 
#Person1#: 아, 알겠어. 그러면 Medicare는 내가 지금 쓸 수 있는 건강 보험은 아니네. 
#Person2#: 맞아. 연방 공제 아래엔 주 공제가 있어. 주 소득세가 있고, 네가 물어본 SUI 세금은 주에서 설립한 실업 및 장애 기금에 기여하는 건데, 굉장히 적은 금액이 공제된다는 걸 알 수 있어. 
#Person1#: 그래, 그거에 1달러 50센트 내는 건 별로 상관없어. 그러니까 주 레벨이랑 연방 레벨, 이렇게 두 가지의 별도 소득세가 있는 거네? 
#Person2#: 맞아. 모든 주가 소득세가 있는 건 아니야. 어떤 주는 더 높은 재산세나 판매세를 사용하지. 
#Person1#: 알겠어. 그 외의 것들은 내가 스스로 해결할 수 있을 것 같아. 건강 보험이랑 내 401(k) 공제는 꽤 명확해. 도와줘서 고마워, Susan. 
#Person2#: 별말을 다해! 그런 공제들이 정말 쌓이고, 아무도 자신의 순 급여가 원하는 만큼 높지 않으니까. 설명이 필요하다니 이해돼. 
#Person1#: 그래, 영국도 똑같은 것 같아. 단지 내가 신경을 별로 안 썼던 것뿐이지. 나중에 봐!


#### 영단어는 항상 Summary에 존재하는가?
- dialogue와 summary에 모두 존재하는 영단어만 추출

In [81]:
two_word_engs = pd.read_csv(os.path.join(project_dir, 'data', "01_two_words_english_counts.csv"))
two_word_engs.columns

Index(['word', 'count', 'ratio'], dtype='object')

In [80]:
merged_df = pd.concat([train_df, val_df], axis=0)
merged_df.shape

(12956, 4)

In [103]:
import regex

# 라틴 문자와 작은따옴표, 마침표를 포함하는 연속된 단어와 구문을 추출합니다.
regex_pattern = r"[\p{Latin}\.']+(?: [\p{Latin}\.']+)*"

text_data = "앙녕ㅎㄴ아ㅓㄹ sdfj 'My Heart Will Go On'., 'bon appétit', the, Karren, Jessica, Mr. Smith, Jack O' Nill"
matches = regex.findall(regex_pattern, text_data)

print(matches)

["sdfj 'My Heart Will Go On'.", "'bon appétit'", 'the', 'Karren', 'Jessica', 'Mr. Smith', "Jack O' Nill"]
